This notebook contains evaluation of LongT5 Model trained on pubmed research paper dataset. ROUGE is used for measuring accuracy. The test example used is from BigPatent dataset.
https://huggingface.co/whaleloops/longt5-tglobal-large-16384-pubmed-10k_steps

This notebook has not been executed so scores are not mentioned.

In [17]:
# installing require packages
!pip install datasets
!pip install transformers
!pip install sentencepiece
!pip install rouge-score

In [ ]:
from datasets import load_dataset

#Mechanical Engineering; Lightning; Heating; Weapons; Blasting
ds = load_dataset("big_patent", "f")  # only 'f' CPC codes


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
#import pandas as pd
# put data into dataFrames and csv
#df_train = pd.DataFrame(ds['train'])
#df_val = pd.DataFrame(ds['validation'])
#df_test = pd.DataFrame(ds['test'])
#df_train.to_csv("train.csv",index=False)
#df_val.to_csv("val.csv",index=False)
#df_test.to_csv("test.csv",index=False)

In [ ]:
# cleaning text
import re

def remove_sentence_with_fig(paragraph):


    # first we remove . after FIG
    paragraph = re.sub(r"FIG.","FIG",paragraph)

    # Split the paragraph into sentences
    sentences = re.split(r'(?<=[.!?])\s+', paragraph)

    # Remove sentences containing "FIG."
    filtered_sentences = [sentence for sentence in sentences if "FIG" not in sentence]

    # Join the remaining sentences to form the updated paragraph
    updated_paragraph = ' '.join(filtered_sentences)

    return updated_paragraph

def clean_text(text):

    # replace ° degree with
    text = re.sub(r"°", " degree", text)

    # replace US Patent
    text = re.sub(r" U.S. Pat. No. [0-9,]+", "Patent", text)

    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)
    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)


    text = text.strip()

    return text


text = ds['test'][0]["description"]
cleaned_text = clean_text(text)
text = remove_sentence_with_fig(cleaned_text)

print(cleaned_text)
print(text)

BACKGROUND OF THE INVENTION This invention relates to a fuel economy running indicator which may be used during the driving operation of an automobile. In general, an internal combustion engine for a vehicle is designed to have a scope for operating at a constant gear ratio of speed change gears even if the load of the vehicle changes considerably. Accordingly, it is possible that a driver selects the gear ratio within the scope of a certain degree to operate a vehicle, such as a truck, having a number of speed change gears. However, under a constant power output, the internal combustion engine used for such a vehicle makes it possible to accomplish a satisfactory state of specific fuel consumption, that is, a fuel economy running state, by setting a suitable operation condition and, for this purpose, it is necessary to continually set a suitable gear ratio or proper travel of an accelerator pedal. This has become a substantial problem in this energy conserving age. Prior to the disclo

In [ ]:
# divide the text into paragraphs of 16000
text1 = text.split()
text_lst = []
for i in range(0,len(text1),16000):
  tmp = " ".join(text1[i:i+16000])
  text_lst.append(tmp)


In [ ]:
# exploring the dataset
print(ds['test'][0]["description"])

print("\n\n\n-------------")
print(ds['test'][0]["abstract"])

BACKGROUND OF THE INVENTION 
     This invention relates to a fuel economy running indicator which may be used during the driving operation of an automobile. 
     In general, an internal combustion engine for a vehicle is designed to have a scope for operating at a constant gear ratio of speed change gears even if the load of the vehicle changes considerably. Accordingly, it is possible that a driver selects the gear ratio within the scope of a certain degree to operate a vehicle, such as a truck, having a number of speed change gears. 
     However, under a constant power output, the internal combustion engine used for such a vehicle makes it possible to accomplish a satisfactory state of specific fuel consumption, that is, a fuel economy running state, by setting a suitable operation condition and, for this purpose, it is necessary to continually set a suitable gear ratio or proper travel of an accelerator pedal. This has become a substantial problem in this energy conserving age. 


In [ ]:
# For cleaning predicted text
def clean_predicted_text(text):


    # replace semi-colon with full-stop
    text = re.sub(r";", ".", text)

    #replace new line character
    text = re.sub(r"<n>", "\n", text)

    #replace start of paragraph
    text = re.sub(r"<s>", "", text)

    # Remove special characters and symbols
    text = re.sub(r"[^a-zA-Z0-9\s.:,-]", "", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)




    return text

In [ ]:
# creating model
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

tokenizer = AutoTokenizer.from_pretrained("whaleloops/longt5-tglobal-large-16384-pubmed-10k_steps")

model = AutoModelForSeq2SeqLM.from_pretrained("whaleloops/longt5-tglobal-large-16384-pubmed-10k_steps")
# summary expected length
#expected_length = len(ds['test'][0]["abstract"])//2

summarizer = pipeline("summarization",model=model,tokenizer=tokenizer)


In [ ]:

#summarize data
#pred_summary_length = expected_length//len(text_lst)
predictions = []
for i in range(0,len(text_lst)):
  tmp = text_lst[i]

  prediction = clean_predicted_text(summarizer(tmp,max_length=500)[0]["summary_text"])
  #predictions
  predictions.append(prediction)


In [ ]:
from rouge_score import rouge_scorer
abstract = ds['test'][0]["abstract"]
predicted_abstract = " ".join(predictions)
scorer = rouge_scorer.RougeScorer(['rouge1','rouge2','rougeL'], use_stemmer=True)
scores = scorer.score(abstract, predicted_abstract)

In [ ]:
predictions

['the present invention provides a fuel economy running indicator which may be used during the driving operation of an automobile . the engine is driven by an internal combustion engine which makes it possible to accomplish a satisfactory state of specific fuel consumption by setting a suitable operation condition and for this purpose it is necessary to continually set a suitable gear ratio or proper travel of an accelerator pedal . this has become a substantial problem in this energy conserving age . in this paper , the invention presents a novel fuel economy running indicator which may be used during the driving operation of an automobile .']

In [ ]:
#rouge max_length tokens 100
scores


{'rouge1': Score(precision=0.42718446601941745, recall=0.2875816993464052, fmeasure=0.34375),
 'rouge2': Score(precision=0.11764705882352941, recall=0.07894736842105263, fmeasure=0.09448818897637795),
 'rougeL': Score(precision=0.24271844660194175, recall=0.16339869281045752, fmeasure=0.1953125)}